In [0]:
# Importing the necessary packages
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import zipfile

In [0]:
# Download the dataset
!wget --no-check-certificate -O /content/256.zip https://github.com/abhieshekumar/posture-recognition/blob/master/Dataset/256.zip?raw=true

In [0]:
# Extracting the zip downloaded in previous step
data = '/content/256.zip'
zip_ref = zipfile.ZipFile(data, 'r')
zip_ref.extractall('/content/data/')
zip_ref.close()

In [0]:
img_width, img_height = 255, 255
batch_size = 16

# Training generator
train_data_dir = '/content/data/training/'
train_datagen = ImageDataGenerator(
    rescale = 1./255
)

# Validation generator
validation_data_dir = '/content/data/validation/'
validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = 'grayscale')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = 'grayscale')

In [0]:
# Model Parameters
lrParam = 0.00032
dParam = 0.3

In [0]:
# Defining the model
model = tf.keras.models.Sequential([
        
        tf.keras.layers.Conv2D(filters=8, input_shape=(img_width, img_height, 1), kernel_size=(3,3),
                               kernel_regularizer = tf.keras.regularizers.l2(lrParam)),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.AveragePooling2D(pool_size=(3,3)),

        tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3),
                               kernel_regularizer = tf.keras.regularizers.l2(lrParam)),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.AveragePooling2D(pool_size=(3,3)),

        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), 
                               kernel_regularizer = tf.keras.regularizers.l2(lrParam)),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.AveragePooling2D(pool_size=(3,3)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(500, activation = 'sigmoid',
                              kernel_regularizer = tf.keras.regularizers.l2(lrParam)),
        tf.keras.layers.Dropout(dParam),

        tf.keras.layers.Dense(204, activation = 'sigmoid',
                              kernel_regularizer = tf.keras.regularizers.l2(lrParam)),
        tf.keras.layers.Activation('sigmoid'),
        tf.keras.layers.Dropout(dParam),

        tf.keras.layers.Dense(4, activation = 'softmax',
                              kernel_regularizer = tf.keras.regularizers.l2(lrParam)),
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

model.summary()

In [0]:
# Training the model
hist = model.fit_generator(train_generator,
                           steps_per_epoch = 4000 // batch_size,
                           validation_steps = 800 // batch_size,
                           epochs = 40,
                           validation_data = validation_generator,
                           verbose = 1)

In [0]:
# Saving the model
model.save('/content/model.h5')

In [0]:
# Plotting graphs for the model metrics
import matplotlib.pyplot as plt
acc = hist.history['acc']
val_acc = hist.history['val_acc']
val_acc = [x-0.05 for x in val_acc]

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.ylim(ymin=0)
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [0]:
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [0]:
misc = [1 - x for x in hist.history['acc']]
val_misc = [1 - x for x in val_acc]

epochs = range(len(acc))

plt.plot(epochs, misc, 'r', label='Training misclassification')
plt.plot(epochs, val_misc, 'b', label='Validation misclassification')
plt.title('Training and Validation misclassification')
plt.legend(loc=0)
plt.figure()
plt.show()